In [2]:
import torch
from isegm.model.modeling.segformer import MixVisionTransformer, SegformerHead

In [3]:
Pretrained_B2_C = '/playpen-raid/qinliu/projects/ritm_interactive_segmentation/weights/pretrained/mit_b2_converted.pth'

backbone_params = dict(
    in_channels=6,
    embed_dims=64,
    num_stages=4,
    num_layers=[3, 4, 6, 3],
    num_heads=[1, 2, 5, 8],
    patch_sizes=[7, 3, 3, 3],
    strides=[2, 2, 2, 2],
    sr_ratios=[8, 4, 2, 1],
    out_indices=(0, 1, 2, 3),
    mlp_ratio=4,
    qkv_bias=True,
    drop_rate=0.0,
    attn_drop_rate=0.0,
    drop_path_rate=0.1,
    pretrained=Pretrained_B2_C
)


In [4]:
backbone = MixVisionTransformer(**backbone_params)
# backbone.init_weights()
state_dict = torch.load(Pretrained_B2_C)
# print(state_dict.keys())


/playpen-raid/qinliu/projects/iSegFormer/isegm/model/modeling/segformer.py:314: UserWarning: DeprecationWarning: pretrained is a deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is a deprecated, '


In [5]:
# remove keys
# state_dict.pop('layers.0.0.projection.weight')
ori_proj_weight = state_dict['layers.0.0.projection.weight']
state_dict['layers.0.0.projection.weight'] = torch.cat([ori_proj_weight, ori_proj_weight], dim=1)


backbone.load_state_dict(state_dict, True)

<All keys matched successfully>

In [6]:
backbone.load_state_dict(state_dict, True)

<All keys matched successfully>

In [7]:
state_dict = torch.load(Pretrained_B2_C)
#print(state_dict.keys())

In [8]:
from isegm.model.modeling.transformer_helper.cross_entropy_loss import CrossEntropyLoss

decode_head_params = dict(
    in_channels=[64, 128, 320, 512],
    in_index=[0, 1, 2, 3],
    channels=256,
    dropout_ratio=0.1,
    num_classes=1,
    loss_decode=CrossEntropyLoss(),
    align_corners=False,
)

head = SegformerHead(**decode_head_params)


In [9]:
in_shape = (1, 3, 320, 480)
input_t = torch.randn(in_shape)

additional_shape = (1, 3, 320, 480)
additional_t = torch.randn(additional_shape)

input_t = torch.cat([input_t, additional_t], dim=1)

temp = backbone(input_t)
out_t = head(temp)

In [11]:
print([temp[i].shape for i in range(len(temp))], out_t.shape)

[torch.Size([1, 64, 160, 240]), torch.Size([1, 128, 80, 120]), torch.Size([1, 320, 40, 60]), torch.Size([1, 512, 20, 30])] torch.Size([1, 1, 160, 240])
